## basic config

In [1]:
import torch
import os
torch.cuda.device_count()

1

In [2]:

def align_faces_2(root_path):
    import os 
    current_path = os.getcwd()
    temp_path = "/home/bean/stylegan3-editing"
    os.chdir(temp_path)
    command =f"""python prepare_data/preparing_faces_parallel.py \
                --mode align \
                --root_path  {root_path} 
                """
    import subprocess
    subprocess.run(command, shell=True)
    os.chdir(current_path)

In [3]:
# DRAGVIDEO_ROOT_PATH = "/Ext_4T_SSD/ASHOK/"
DRAGVIDEO_ROOT_PATH = "/media/Ext_4T_SSD/ASHOK/"

# keep matching encoder,SG as pairt 
old = {"encoder":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/e4e_ffhq_encode.pt",
       "SG": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/ffhq.pkl",
       "model_name":"stylegan2",
       "resolution":1024,
       }

new = {"encoder": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/restyle_e4e_ffhq.pt",
         "SG":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/stylegan3_3rdtime_weights/stylegan3-r-ffhq-1024_module.pkl",
         "model_name":"stylegan3",
               "resolution":1024,
         }

new_256 = {"encoder": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/restyle_e4e_ffhq.pt",
         "SG":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/Stylegan3_1_files/stylegan3-r-ffhqu-256x256.pkl",
         "model_name":"stylegan3",
         "resolution":256,
         }

videos = {
       "obama": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/obama.mp4",
       "man_speaking": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/person_speaking_original.mp4",
       "rahul": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/rahul.mp4",
       "alien_girl": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/alien_girl.mp4",
       "mirrAR": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/mirrAR.mp4",
       "vsauce_frnd":f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/micheal_friendzone.mp4",
       "actress_tan": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/gadot_tanned_left.mp4",
       "woman_waiting":f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/raw_videos/woman_waiting.mp4"
}

In [4]:
EXPERIMENT_NAME_POSTFIX = "woman_waiting"#"aligned_sg3_&_new_e4e_reconstruction_invstep_200_pti_step_450"
models_set = old
Testing =True
# --------------------------------  


import datetime
lazy_config = {
    
    #-----------------
    # often changed
    #-----------------
    "video_path":  videos["woman_waiting"],
    "n_frames" : 150,#120,#200,
    "N_STEPS_in_draggan":  "200",
    "editing_function_name":"make_jaw_wider",#"smile", #"smile",#"large_eyes",#"large_eyes" # "make_jaw_wider" # "mouth_wide","up_eyebrows"
        
    # less often changed
    "EXP_NAME": str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+"_"+ EXPERIMENT_NAME_POSTFIX,
    "e4e": models_set['encoder'],
    "stylegan2_ada_ffhq": models_set['SG'],
    "model_name" : models_set['model_name'],
    "IMAGE_SIZE": models_set["resolution"],
    "fps": 24,
    
}

env_config = {
    "DragGAN_dir": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN",
    "Experiment_base_path":f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/experiments/" ,
    "init_exp_dir_shell_path": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/init_datadirs.sh",
    "dummy_config_path" : f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/configs/dummy",
    
    
}

#hyper parameters in PTI

hyper_config = {
    "max_pti_steps": 350,
    "first_inv_steps":450,
    "max_images_to_invert": 200,
}

if Testing: 
    hyper_config["max_pti_steps"] = 1#200
    hyper_config["first_inv_steps"] = 1#350
    hyper_config["max_images_to_invert"] = 5
    lazy_config["N_STEPS_in_draggan"] = '150'
    lazy_config["n_frames"] = 100
    lazy_config["editing_function_name"]="smile"
    


# ----------------------------------------------------------------------------------------------


# create experiment data folder structure
Experiment_name = lazy_config["EXP_NAME"]
Experiment_base_path = env_config["Experiment_base_path"]
Experiment_path = os.path.join(Experiment_base_path, Experiment_name)

# change path configs , hyperparameters 
paths_config_dict = {
    #pretrained models
    "e4e": lazy_config["e4e"],
    "stylegan2_ada_ffhq": lazy_config["stylegan2_ada_ffhq"],
    
    # to store tuned stylegan weights
    "checkpoints_dir": os.path.join(Experiment_path,'tuned_SG'),
    # to store latents
    "embedding_base_dir": os.path.join(Experiment_path,'latents'),
    # aligned / processed images
    "input_data_path": os.path.join(Experiment_path,'aligned'),
     "quad_values_path": os.path.join(Experiment_path,'quad_values'),
     "dlib" : f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/align.dat"
     
}



In [5]:
from run_utils import *

init_experiment_dir(Experiment_name,Experiment_base_path,shell_script_path=env_config["init_exp_dir_shell_path"])

# dummy paths_config overwrites the paths_config.py
# dummy_config_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/configs/dummy'
add_dummy_config_from_dict("hyperparameters.py", hyper_config,ROOT_PATH=env_config["dummy_config_path"])

add_dummy_config_from_dict("paths_config.py", paths_config_dict,ROOT_PATH=env_config["dummy_config_path"])



# add all these configs to log.txt
# --------------------------------  
with open(os.path.join(Experiment_path,'log.txt'), 'a') as f:
    import json
    f.write(f"lazy_config: {json.dumps(lazy_config, indent=4)}\n")
    f.write(f"env_config: {json.dumps(env_config, indent=4)}\n")
    f.write(f"hyper_config: {json.dumps(hyper_config, indent=4)}\n")
    
    



In [6]:
# extract frames from video
#==============================================================================

from importlib.machinery import SourceFileLoader
# imports the module from the given path
# video_utils = SourceFileLoader("video_utils","../utils_draggan/video_utils.py").load_module()

ffmpeg_utils = SourceFileLoader("video_utils","../utils_draggan/ffmpeg_utils.py").load_module()
raw_path = os.path.join(Experiment_path, "raw")

# # video_utils.extract_frames(lazy_config['video_path'], raw_path,n_frames=lazy_config['n_frames'])
ffmpeg_utils.ffmpeg.extract_frames(video_path = lazy_config['video_path'],
                                      output_frames_dir = raw_path,
                                      fps = lazy_config['fps'],
                                      first_n_frames = lazy_config['n_frames'],
)

ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/opt/conda/conda-bld/ffmpeg_1597178665428/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libsw

# start

In [7]:
# from utils.align_data import pre_process_images
# pre_process_images(raw_path, IMAGE_SIZE=lazy_config['IMAGE_SIZE']) # o/p: config.input_data_path ,quad pickle dir
# #
# # align_faces_2(os.path.join(Experiment_path,'raw'))

In [8]:
#doc: Experiment_path should have raw folder with images
# it creates inputs folder with 5 subfolders and moves images from raw to inputs


parts = 50
#------------------------------------------------------------
raw_images_path = os.path.join(Experiment_path,'raw')
os.makedirs(os.path.join(Experiment_path,'inputs'),exist_ok=True)

for i in range(parts):
    os.makedirs(os.path.join(Experiment_path,'inputs',f"part_{i}"),exist_ok=True)
    #raw folder inside each part
    os.makedirs(os.path.join(Experiment_path,'inputs',f"part_{i}",'raw'),exist_ok=True)
    
raw_images = os.listdir(raw_images_path)
count = len(raw_images)
#split raw images into 5 parts
raw_parts = [ raw_images[i:i + count//parts] for i in range(0, count, count//parts) ]
for i in range(parts):
    for img in raw_parts[i]:
        # os.rename(os.path.join(raw_images_path,img),os.path.join(Experiment_path,'inputs','part_{i}','raw',img))
        os.rename(os.path.join(raw_images_path,img),os.path.join(Experiment_path,'inputs',f"part_{i}",'raw',img))
        

ValueError: range() arg 3 must not be zero

In [ ]:
# # run 5 processes in parallel ( _02_run_PTI.py)
# #==============================================================================
# import subprocess
# import os
# import time

# for i in range(1,6):
#     os.chdir(f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI")
#     command =f"""python _02_run_PTI.py /media/Ext_4T_SSD/ASHOK/DragVideo/Data_store/experiments/2023-09-17_23-04-13_woman_waiting/inputs/part_0 """ 
#     subprocess.Popen(command, shell=True)
#     time.sleep(1)